app for resnet50

In [ ]:
from flask import Flask, render_template, request
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os

# ===== Flask App =====
app = Flask(__name__)
UPLOAD_FOLDER = "static"
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

# ===== Class Names =====
CLASS_NAMES = ['adenosis', 'ductal_carcinoma', 'fibroadenoma', 'lobular_carcinoma',
               'mucinous_carcinoma', 'papillary_carcinoma', 'phyllodes_tumor', 'tubular_adenoma']

# ===== Image Transform =====
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ===== Load Model =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=None)  # No pretrained weights, we'll load our own
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(CLASS_NAMES))
model.load_state_dict(torch.load("model3.pth", map_location=device))
model.to(device)
model.eval()

def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    return CLASS_NAMES[predicted.item()]

# ===== Routes =====
@app.route("/", methods=["GET", "POST"])
def index():
    prediction = None
    image_path = None

    if request.method == "POST":
        if "file" not in request.files:
            return render_template("index.html", prediction="No file uploaded")

        file = request.files["file"]
        if file.filename == "":
            return render_template("index.html", prediction="No file selected")

        # Save file
        image_path = os.path.join(app.config["UPLOAD_FOLDER"], file.filename)
        file.save(image_path)

        # Predict
        prediction = predict_image(image_path)

    return render_template("index.html", prediction=prediction, image_path=image_path)

if __name__ == "__main__":
    app.run(debug=True)


index.html for resnet 50

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Breast Cancer Classification</title>
</head>
<body>
    <h2>Upload an Image for Classification</h2>
    <form action="/" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*" required>
        <input type="submit" value="Upload and Predict">
    </form>

    {% if prediction %}
        <h3>Prediction: {{ prediction }}</h3>
        <img src="{{ image_path }}" alt="Uploaded Image" width="300">
    {% endif %}
</body>
</html>

basic app.py for efficientnet

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from flask import Flask, request, render_template, jsonify
from PIL import Image
import os

# ----------------------------
# Flask app initialization
# ----------------------------
app = Flask(__name__)

# ----------------------------
# Device setup
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Define class names (same as training order)
# ----------------------------
CLASS_NAMES = [
    'adenosis', 'ductal_carcinoma', 'fibroadenoma',
    'lobular_carcinoma', 'mucinous_carcinoma',
    'papillary_carcinoma', 'phyllodes_tumor', 'tubular_adenoma'
]

# ----------------------------
# Load EfficientNet model
# ----------------------------
def load_model(weights_path="model3.pth"):
    model = models.efficientnet_b0(weights=None)   # change to efficientnet_b1/b2 if trained with that
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, len(CLASS_NAMES))

    model.load_state_dict(torch.load(weights_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

model = load_model()

# ----------------------------
# Image preprocessing
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    img_t = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_t)
        _, preds = torch.max(outputs, 1)
        predicted_class = CLASS_NAMES[preds.item()]
        confidence = torch.nn.functional.softmax(outputs, dim=1)[0][preds.item()].item()

    return predicted_class, confidence

# ----------------------------
# Flask Routes
# ----------------------------
@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        if "file" not in request.files:
            return "No file uploaded", 400

        file = request.files["file"]
        if file.filename == "":
            return "No file selected", 400

        file_path = os.path.join("uploads", file.filename)
        os.makedirs("uploads", exist_ok=True)
        file.save(file_path)

        pred_class, confidence = predict_image(file_path)
        return render_template("index.html", prediction=pred_class, confidence=confidence)

    return render_template("index.html")

# ----------------------------
# Run Flask app
# ----------------------------
if __name__ == "__main__":
    app.run(debug=True)


index.html for efficient net

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Breast Cancer Classification</title>
</head>
<body>
    <h2>Upload a Histopathology Image</h2>
    <form method="post" enctype="multipart/form-data">
        <input type="file" name="file" required>
        <input type="submit" value="Predict">
    </form>

    {% if prediction %}
        <h3>Prediction: {{ prediction }}</h3>
        <h4>Confidence: {{ confidence|round(3) }}</h4>
    {% endif %}
</body>
</html>